MAE 207 - Bioinspired Robotics
| HW 1
| James Salem

In [169]:
# symbolic computation tools
import sympy
from sympy import symbols, pprint
from sympy import sin, cos, asin, acos, pi, sqrt
from sympy import Matrix

import numpy as np

In [170]:
def T(theta, x, y):
    """
    Function to return an arbitrary transformation matrix 
    This is for sympy symbolic calculation
    """
    return Matrix([[cos(theta), -sin(theta), x], 
                   [sin(theta), cos(theta), y],
                   [0, 0, 1]])

def sym_to_np(T):
    return np.array(T).astype(np.float64)

Problem 1

Compute the forward kinematics, solving for the foot position (x, y) as a function of
motor angles, for both the serial and parallel leg configurations.

Serial Configuration

<img src="MAE 207 HW1 Serial.png">

In [171]:
# Define symbols
(theta1_sym, 
 theta2_sym, 
 link1_sym, 
 link2_sym) = symbols("""theta1_sym 
                         theta2_sym 
                         link1_sym 
                         link2_sym""" , real = True)

In [172]:
# Make Transormation matrices
T_01 = T(theta1_sym, 0, 0)
T_12 = T(theta2_sym, link1_sym, 0)
print("T_01 = ")
pprint(T_01)
print("T_12 = ")
pprint(T_12)

T_01 = 
⎡cos(θ_1_sym)  -sin(θ_1_sym)  0⎤
⎢                              ⎥
⎢sin(θ_1_sym)  cos(θ_1_sym)   0⎥
⎢                              ⎥
⎣     0              0        1⎦
T_12 = 
⎡cos(θ_2_sym)  -sin(θ_2_sym)  link_1_sym⎤
⎢                                       ⎥
⎢sin(θ_2_sym)  cos(θ_2_sym)       0     ⎥
⎢                                       ⎥
⎣     0              0            1     ⎦


In [173]:
# Multiply transformation matrices together
T_02 = sympy.simplify(T_01*T_12)

print("T_02 = ")
pprint(T_02)

T_02 = 
⎡cos(θ_1_sym + θ_2_sym)  -sin(θ_1_sym + θ_2_sym)  link_1_sym⋅cos(θ_1_sym)⎤
⎢                                                                        ⎥
⎢sin(θ_1_sym + θ_2_sym)  cos(θ_1_sym + θ_2_sym)   link_1_sym⋅sin(θ_1_sym)⎥
⎢                                                                        ⎥
⎣          0                        0                        1           ⎦


In [174]:
# Calculate position of foot in reference frame #0
xy_pos = T_02 * Matrix([[link2_sym],[0],[1]])
print("(x, y, 1) = ")
pprint(xy_pos)

(x, y, 1) = 
⎡link_1_sym⋅cos(θ_1_sym) + link_2_sym⋅cos(θ_1_sym + θ_2_sym)⎤
⎢                                                           ⎥
⎢link_1_sym⋅sin(θ_1_sym) + link_2_sym⋅sin(θ_1_sym + θ_2_sym)⎥
⎢                                                           ⎥
⎣                             1                             ⎦


In [175]:
x = xy_pos[0]
print('x =', x)

y = xy_pos[1]
print('y =', y)

x = link1_sym*cos(theta1_sym) + link2_sym*cos(theta1_sym + theta2_sym)
y = link1_sym*sin(theta1_sym) + link2_sym*sin(theta1_sym + theta2_sym)


Parallel Configuration

<img src="MAE 207 HW1 Par.png">

In [176]:
# Make Transformation matrices
T_01_par = T((theta2_sym + theta1_sym) / 2, 0, 0)
pprint(T_01_par)

⎡   ⎛θ_1_sym   θ_2_sym⎞      ⎛θ_1_sym   θ_2_sym⎞   ⎤
⎢cos⎜─────── + ───────⎟  -sin⎜─────── + ───────⎟  0⎥
⎢   ⎝   2         2   ⎠      ⎝   2         2   ⎠   ⎥
⎢                                                  ⎥
⎢   ⎛θ_1_sym   θ_2_sym⎞     ⎛θ_1_sym   θ_2_sym⎞    ⎥
⎢sin⎜─────── + ───────⎟  cos⎜─────── + ───────⎟   0⎥
⎢   ⎝   2         2   ⎠     ⎝   2         2   ⎠    ⎥
⎢                                                  ⎥
⎣          0                        0             1⎦


In [177]:
# Calculate position of foot in reference frame #1
y1 = 0
h1 = link1_sym * cos((theta2_sym - theta1_sym) / 2)
h2 = sqrt(link2_sym**2 - link1_sym**2 * sin((theta2_sym - theta1_sym) / 2)**2)
x1 = h1 + h2

In [178]:
# Calculate position of foot in reference frame #0
xy_pos_par = T_01_par * Matrix([[x1],[y1],[1]])
print("(x0, y0) = ")
pprint(xy_pos_par[:2,:])

(x0, y0) = 
⎡⎛                                        ____________________________________
⎢⎜              ⎛θ_1_sym   θ_2_sym⎞      ╱             2    2⎛θ_1_sym   θ_2_sy
⎢⎜link_1_sym⋅cos⎜─────── - ───────⎟ +   ╱  - link_1_sym ⋅sin ⎜─────── - ──────
⎢⎝              ⎝   2         2   ⎠   ╲╱                     ⎝   2         2  
⎢                                                                             
⎢⎛                                        ____________________________________
⎢⎜              ⎛θ_1_sym   θ_2_sym⎞      ╱             2    2⎛θ_1_sym   θ_2_sy
⎢⎜link_1_sym⋅cos⎜─────── - ───────⎟ +   ╱  - link_1_sym ⋅sin ⎜─────── - ──────
⎣⎝              ⎝   2         2   ⎠   ╲╱                     ⎝   2         2  

_________________⎞                       ⎤
m⎞             2 ⎟    ⎛θ_1_sym   θ_2_sym⎞⎥
─⎟ + link_2_sym  ⎟⋅cos⎜─────── + ───────⎟⎥
 ⎠               ⎠    ⎝   2         2   ⎠⎥
                                         ⎥
_________________⎞                       ⎥
m⎞             2 ⎟

Problem 2 

Compute the Jacobian for both the serial and parallel leg configurations.

Serial Configuration

In [179]:
# compute Jacobian for Serial Configuration
J = xy_pos[:2,:].jacobian([theta1_sym, theta2_sym])
print("Jacobian Matrix")
pprint(J, wrap_line = False)

Jacobian Matrix
⎡-link_1_sym⋅sin(θ_1_sym) - link_2_sym⋅sin(θ_1_sym + θ_2_sym)  -link_2_sym⋅sin(θ_1_sym + θ_2_sym)⎤
⎢                                                                                                ⎥
⎣link_1_sym⋅cos(θ_1_sym) + link_2_sym⋅cos(θ_1_sym + θ_2_sym)   link_2_sym⋅cos(θ_1_sym + θ_2_sym) ⎦


Parallel Configuration

In [180]:
# compute Jacobian for Parallel Configuration
J_par = xy_pos_par[:2,:].jacobian([theta1_sym, theta2_sym])
print("Jacobian Matrix")
print(J_par)

Jacobian Matrix
Matrix([[-(link1_sym*cos(theta1_sym/2 - theta2_sym/2) + sqrt(-link1_sym**2*sin(theta1_sym/2 - theta2_sym/2)**2 + link2_sym**2))*sin(theta1_sym/2 + theta2_sym/2)/2 + (-link1_sym**2*sin(theta1_sym/2 - theta2_sym/2)*cos(theta1_sym/2 - theta2_sym/2)/(2*sqrt(-link1_sym**2*sin(theta1_sym/2 - theta2_sym/2)**2 + link2_sym**2)) - link1_sym*sin(theta1_sym/2 - theta2_sym/2)/2)*cos(theta1_sym/2 + theta2_sym/2), -(link1_sym*cos(theta1_sym/2 - theta2_sym/2) + sqrt(-link1_sym**2*sin(theta1_sym/2 - theta2_sym/2)**2 + link2_sym**2))*sin(theta1_sym/2 + theta2_sym/2)/2 + (link1_sym**2*sin(theta1_sym/2 - theta2_sym/2)*cos(theta1_sym/2 - theta2_sym/2)/(2*sqrt(-link1_sym**2*sin(theta1_sym/2 - theta2_sym/2)**2 + link2_sym**2)) + link1_sym*sin(theta1_sym/2 - theta2_sym/2)/2)*cos(theta1_sym/2 + theta2_sym/2)], [(link1_sym*cos(theta1_sym/2 - theta2_sym/2) + sqrt(-link1_sym**2*sin(theta1_sym/2 - theta2_sym/2)**2 + link2_sym**2))*cos(theta1_sym/2 + theta2_sym/2)/2 + (-link1_sym**2*sin(theta1_sym/2

In [181]:
print(J_par.shape)

(2, 2)


Problem 3

Compute the motor torques required for the robot to stand in place in the configurations
given in the table below. Assume each leg is supporting 1.5 kg.

In [182]:
# Force vector
F = np.array([[0, 1.5 * 9.8]]).T

Serial Configuration : l1 = 10cm, l2 = 10cm, theta1 = 135 degrees, theta2 = 270 degrees

In [183]:
# Values to plug in
l1 = 0.1
l2 = 0.1
theta1 = 135*np.pi/180
theta2 = 270*np.pi/180

In [184]:
# Plug in values into Jacobian
J_real = sym_to_np(J.subs([(theta1_sym, theta1),(theta2_sym, theta2), (link1_sym, l1), (link2_sym, l2)]))

In [185]:
# calculate Torque required to apply force, F
Torque = J_real.T @ F

print("Torque(N*m) Motor 1:", Torque[0])
print("Torque(N*m) Motor 2:", Torque[1])

Torque(N*m) Motor 1: [2.04003481e-16]
Torque(N*m) Motor 2: [1.03944697]


Parallel Configuration: l1 = 10cm, l2 = 10cm, theta1 = 45 degrees, theta2 = 135 degrees

In [186]:
# Values to plug in
l1 = 0.1
l2 = 0.1
theta1 = 45*np.pi/180
theta2 = 135*np.pi/180

In [187]:
# Plug in values into Jacobian
J_real_par = sym_to_np(J_par.subs([(theta1_sym, theta1),(theta2_sym, theta2), (link1_sym, l1), (link2_sym, l2)]))

In [188]:
# calculate Torque required to apply force, F
Torque_par = J_real_par.T @ F

print("Torque(N*m) Motor 1:", Torque_par[0])
print("Torque(N*m) Motor 2:", Torque_par[1])

Torque(N*m) Motor 1: [1.03944697]
Torque(N*m) Motor 2: [-1.03944697]
